In [18]:
"""
FUNDAMENTAL
TECHNICAL 
    (MOVING_AVERAGE)
TRADER
"""

'\nFUNDAMENTAL\nTECHNICAL \n    (MOVING_AVERAGE)\nTRADER\n'

In [19]:
import datetime as dt
import pandas as pd

from de.processor.dart import DART_FUNDAMENTAL_PROCESSOR
from de.processor.fdr import FDR_INFO_PROCESSOR, FDR_OHLCV_PREPROCESSOR
from de.loader.pykrx_loader import get_pykrx_loader

from de.processor.factor.fundamental import FUNDAMENTAL_FACTOR_PROCESSOR
from de.processor.factor.technical import MOVING_AVERAGE_FACTOR_PROCESSOR
from de.processor.factor.trader import TRADER_FACTOR_PROCESSOR

from de.model_paramter import MODEL_PARAMTER_PROCESSOR

from de.analyser.factor import FACTOR_ANALYSER
from de.processor.factor.filter import FACTOR_FILTER

In [20]:
CFG = {
    "FUNDAMENTAL": {
        "PROCESSOR": {
            "ohlcv_recent_n": 10,
        },
        "FILTER": {},
    },
    "MOVING_AVERAGE": {
        "PROCESSOR": {
            "factors": ["Close", "VolumeRotation"],
            "short_term_window": round((7 * 2) * (5 / 7)),
            "long_term_window": round((7 * 8) * (5 / 7)),
            "signal_recent_n": 20,
        },
        "FILTER": {},
    },
    "TRADER": {
        "PROCESSOR": {
            "start": (dt.date.today() - dt.timedelta(days=9)).strftime("%Y-%m-%d"),
            "end": (dt.date.today() - dt.timedelta(days=1)).strftime("%Y-%m-%d"),
            # "end": dt.date.today().strftime("%Y-%m-%d"),
        },
        "FILTER": {
            "absolute_conditions": [
                "Corp > 0",
                "Foreign > 0",
                "Indivisual < 0",
            ]
        },
    },
    "FACTOR_ANALYSIS": {
        "profit_recent_n": 20,
    },
}
recen_n = CFG["FACTOR_ANALYSIS"]["profit_recent_n"]
analysis_date = (dt.date.today() - dt.timedelta(days=recen_n)).strftime("%Y-%m-%d")
CFG["analysis_date"] = analysis_date

In [21]:
# LOAD

In [22]:
# dart_fundamental
dart_fundamental_df = pd.read_csv("./de/data/dart_fundamental_df.csv", index_col=0)
dart_fundamental_processor = DART_FUNDAMENTAL_PROCESSOR(dart_fundamental_df)

In [23]:
# fdr_info
fdr_info_df = pd.read_csv("./de/data/fdr_info_df.csv", index_col=0)
fdr_info_processor = FDR_INFO_PROCESSOR(fdr_info_df)

In [24]:
# fdr_ohlcv
fdr_ohlcv_df = pd.read_csv("./de/data/fdr_ohlcv_df.csv", index_col=0)
fdr_ohlcv_preprocessor = FDR_OHLCV_PREPROCESSOR(fdr_ohlcv_df)
ohlcv_df = fdr_ohlcv_preprocessor.get_pps_fdr_ohlcv_df()

In [25]:
# split ohlcv
future_ohlcv_df = ohlcv_df[CFG["analysis_date"] < ohlcv_df["Date"]].copy()
past_ohlcv_df = ohlcv_df[ohlcv_df["Date"] < CFG["analysis_date"]].copy()

In [26]:
model_paramter_processor = MODEL_PARAMTER_PROCESSOR(CFG["FACTOR_ANALYSIS"])

In [27]:
# Fundamental

## Parameter
### 1. Get Factor_df
fundamental_factor_processor = FUNDAMENTAL_FACTOR_PROCESSOR(
    past_ohlcv_df, dart_fundamental_processor, fdr_info_processor
)
fundamental_factors_df = fundamental_factor_processor.get_fundamental_factor_df(CFG["FUNDAMENTAL"]["PROCESSOR"])

### 2. Get Parameter
fundamental_factor_analyser = FACTOR_ANALYSER(fundamental_factors_df)
fundamental_params_dict = model_paramter_processor.get_params_dict(future_ohlcv_df, fundamental_factor_analyser)
fundamental_param_df = pd.DataFrame().from_dict(fundamental_params_dict, orient="index", columns=["Value", "Variance"])

### 3. Set Paramter
fundamental_param = fundamental_param_df.nlargest(10, "Value").nsmallest(1, "Variance").index[0]
CFG["FUNDAMENTAL"]["FILTER"]["quantile_conditions"] = fundamental_param

## Model
### 1. Get Factor_df
fundamental_factor_processor = FUNDAMENTAL_FACTOR_PROCESSOR(ohlcv_df, dart_fundamental_processor, fdr_info_processor)
fundamental_factor_df = fundamental_factor_processor.get_fundamental_factor_df(CFG["FUNDAMENTAL"]["PROCESSOR"])

### 2. Get Filtered_stockcodes
factor_filter = FACTOR_FILTER(fundamental_factor_df)
fundamental_stockcodes = factor_filter.get_filtered_stockcodes(CFG["FUNDAMENTAL"]["FILTER"])

In [28]:
fundamental_param_df.nlargest(5, "Value").nsmallest(1, "Variance")

,Value,Variance
"((CEPP, 0.0, 0.3), (TLR, 0.3, 0.6))",0.051778,0.001322


In [29]:
# Technical

## Paramter
### 1. Get Factor_df
moving_average_factor_processor = MOVING_AVERAGE_FACTOR_PROCESSOR(past_ohlcv_df, fdr_info_processor)
moving_average_factors_df = moving_average_factor_processor.get_moving_average_factor_df(
    CFG["MOVING_AVERAGE"]["PROCESSOR"]
)

### 2. Get Parameter
moving_average_factor_analyser = FACTOR_ANALYSER(moving_average_factors_df)
moving_average_params_dict = model_paramter_processor.get_params_dict(future_ohlcv_df, moving_average_factor_analyser)
moving_average_param_df = pd.DataFrame().from_dict(
    moving_average_params_dict, orient="index", columns=["Value", "Variance"]
)

### 3. Set Paramter
moving_average_param = moving_average_param_df.nlargest(10, "Value").nsmallest(1, "Variance").index[0]
CFG["MOVING_AVERAGE"]["FILTER"]["quantile_conditions"] = moving_average_param

## Model
### 1. Get Factor_df
moving_average_factor_processor = MOVING_AVERAGE_FACTOR_PROCESSOR(ohlcv_df, fdr_info_processor)
moving_average_factors_df = moving_average_factor_processor.get_moving_average_factor_df(CFG["MOVING_AVERAGE"]["PROCESSOR"])

### 2. Get Filtered_stockcodes
factor_filter = FACTOR_FILTER(moving_average_factors_df)
moving_average_stockcodes = factor_filter.get_filtered_stockcodes(CFG["MOVING_AVERAGE"]["FILTER"], fundamental_stockcodes)

In [30]:
moving_average_param_df.nlargest(5, "Value").nsmallest(1, "Variance")

,Value,Variance
"((VolumeRotation, 0.5, 0.8), (Close, 0.3, 0.6))",0.042696,0.000892


In [31]:
moving_average_stockcodes

{'002450',
 '007720',
 '014190',
 '035420',
 '044780',
 '067900',
 '073560',
 '221980'}

In [32]:
# Trader
pykrx_loader = get_pykrx_loader()

## 1. Get Factor_df
trader_factor_processor = TRADER_FACTOR_PROCESSOR(pykrx_loader)
trader_factor_df = trader_factor_processor.get_trader_factor_df(moving_average_stockcodes, CFG["TRADER"]["PROCESSOR"])

## Model
factor_filter = FACTOR_FILTER(trader_factor_df)
final_stockcodes = factor_filter.get_filtered_stockcodes(CFG["TRADER"]["FILTER"], moving_average_stockcodes)

In [34]:
trader_factor_df

,StockCode,Corp,Indivisual,Foreign
0,007720,0.000000e+00,-9.757474e+05,1.094108e+06
1,073560,9.616600e+03,1.837100e+07,-1.199318e+07
2,002450,2.169086e+05,5.859354e+07,-7.833359e+06
3,014190,-1.532665e+06,1.335390e+08,-1.240649e+08
4,067900,-8.134655e+08,4.914931e+08,3.020707e+08
5,035420,-9.221779e+09,-4.014346e+09,1.998966e+10
6,221980,8.207126e+06,6.536872e+06,-1.534404e+07
7,044780,9.390772e+06,3.882431e+06,-1.329676e+07


In [33]:
final_stockcodes

set()